## Loading libraries

In [13]:
from collections import Counter
import matplotlib.pyplot as plt
import keras

## Building the game

In [14]:
%matplotlib inline
import math
import random
import matplotlib.pyplot as plt
import time
from IPython import display
import numpy as np

class Snake:
    def __init__(self, width=50, height=50, length=4, skipgui=False, renderdelay=0.2):
        assert length < width, "length is less than width"
        assert length < height, "length is less than height"
        self.width = width
        self.height = height
        self.length = length
        self.skipgui = skipgui
        self.renderdelay = renderdelay
        self.snake_generate()
        self.food_generate()
        self.done=False
        
    def snake_generate(self):
        x = random.randint(self.length,self.width-self.length)
        y = random.randint(self.length,self.height-self.length)
        vertical=random.randint(0,2)
        direction=random.choice([-1,1])
        self.snake = [[x,y]]
        for _ in range(self.length-1):
            if vertical == 1:
                self.snake.append([self.snake[-1][0],self.snake[-1][1]+direction])
            else:
                self.snake.append([self.snake[-1][0]+direction,self.snake[-1][1]])
        #self.snake = [[x,int(self.height/2)] for x in range(0,self.length)]
        
    def food_generate(self):
        while True: 
            self.food_loc = [random.randint(0,self.width-1),random.randint(0,self.height-1)]
            if self.food_loc not in self.snake:
                break
                
    def step(self,directions):
        for direction in directions:
            assert self.done==False,"Game ended!!"
            self.move(direction)
            self.done= not self.ifsurvived()
            self.iffoodeaten()
            if not self.skipgui:
                self.rendergui()
            display.clear_output(wait=True)

    def move(self,direction):
        new = self.snake[-1].copy()
        assert direction<4, "Wrong direction"
        if direction == 0: #RIGHT
            new[0] += 1
        elif direction == 1: #UP
            new[1] += 1
        elif direction == 2: #LEFT
            new[0] -= 1
        else: #DOWN
            new[1] -= 1
        self.snake.append(new)        
        
    def ifsurvived(self):
        if self.snake[-1][0]<0 or self.snake[-1][1]<0 or \
        self.snake[-1][0]==self.width or self.snake[-1][1]==self.height or \
        self.snake[-1] in self.snake[:-1]:
            return False
        else:
            return True
    
    def iffoodeaten(self):
        if self.food_loc in self.snake:
            self.food_generate()
            return True
        else:
            self.snake.pop(0)
            return False
        
    def rendergui(self):
        display.clear_output(wait=True)
        display.display(self.plotenv())
        time.sleep(self.renderdelay)
        
        
    def plotenv(self):
        plt.clf()
        plt.plot([x[0] for x in self.snake[:-1]],[x[1] for x in self.snake[:-1]], 'bo')
        plt.plot(self.snake[-1][0],self.snake[-1][1], 'go')
        plt.plot(self.food_loc[0],self.food_loc[1], 'ro')
        plt.axis((0,self.width,0,self.height))
        return plt.gcf()
    
    def direction(self):
        return [x-y for (x,y) in zip(self.snake[-1],self.snake[-2])]

    def leftDirection(self):
        d = self.direction()
        return [-d[1],d[0]]
    
    def rightDirection(self):
        l = self.leftDirection()
        return [-l[0],-l[1]]
    
    def stepFLR(self,directions):
        for direction in directions:
            assert self.done==False,"Game ended!!"
            self.moveFLR(direction)
            self.done= not self.ifsurvived()
            self.iffoodeaten()
            if not self.skipgui:
                self.rendergui()
            display.clear_output(wait=True)

    def moveFLR(self,direction):
        new = self.snake[-1].copy()
        if direction == 0: #FORWARD
            d = self.direction()
        elif direction == 1: #RIGHT
            d = self.rightDirection()
        elif direction == -1: #LEFT
            d = self.leftDirection()
        else:
            assert 1==1, 'Wrong direction'
        new[0] += d[0]
        new[1] += d[1]
        self.snake.append(new)
        
    def appleDirection(self):
        head = self.snake[-1]
        return [head[0]-self.food_loc[0],head[1]-self.food_loc[1]]
    
    def angleBtw(self):
        sd = np.array(self.direction())
        ad = np.array(self.appleDirection())
        #convert to unit vectors
        sd = sd / np.linalg.norm(sd)
        ad = ad / np.linalg.norm(ad)
        #return np.arccos(np.dot(sd, ad))
        return math.atan2(sd[0] * ad[1] - sd[1] * ad[0], sd[0] * ad[0] + sd[1] * ad[1]) / math.pi
        
    def parameters(self):
        ifobstacle = []
        for i in [-1,0,1]:
            self.moveFLR(i)
            ifobstacle.append(not self.ifsurvived())
            self.moveFLR(i)
            ifobstacle.append(not self.ifsurvived())
            self.snake.pop(-1)
            self.snake.pop(-1)
        ifobstacle.append(self.angleBtw())
        return ifobstacle
        
    def foodDistance(self):
        #Manhattan distance
        return abs(self.snake[-1][0]-self.food_loc[0])+abs(self.snake[-1][1]-self.food_loc[1])


## DataSet Generator

In [33]:
import pandas as pd
class SnakeGameDatasetGenerator:
    def __init__(self,Ngames=10,Nsteps=100):
        self.Ngames = Ngames
        self.Nsteps = Nsteps
        self.dataset = []
        for _ in range(Ngames):
            self.playGame()
        
        
    def playGame(self):
        snake = Snake(skipgui=True)
        prevscore = len(snake.snake)
        prevfooddist = snake.foodDistance()
        for direction in [random.randint(-1,1) for _ in range(self.Nsteps)]:
            observation = snake.parameters()
            observation.append(direction)
            snake.stepFLR([direction])
            newscore = len(snake.snake)
            newfooddist = snake.foodDistance()
            if snake.done:
                observation.append(-1)
            elif newscore > prevscore or newfooddist<prevfooddist:
                observation.append(1)
            else:
                observation.append(0)
            prevscore=newscore
            prevfooddist=newfooddist
            self.dataset.append(observation)
            if snake.done:
                break 
    def saveDataset(self,features=['IsLeftObstacle','IsLeftLeftObstacle','IsForwardObstacle','IsForwardForwardObstacle','IsRightObstacle','IsRightRightObstacle','AngleWithFood','Direction','Decision']):
        datasetdf = pd.DataFrame(dg.dataset,columns=features)
        filename = 'SnakegameDataset2_'+str(self.Ngames)+'_'+str(self.Nsteps)+'.csv'
        datasetdf.to_csv(filename)
        

In [34]:
dg = SnakeGameDatasetGenerator(Ngames=1000,Nsteps=1000)
print(len(dg.dataset))

from collections import Counter
freq = Counter([x[4] for x in dg.dataset])
print(freq)

dg.saveDataset()

17477
Counter({False: 16054, True: 1423})


## Model Building

In [17]:
import pandas as pd

In [18]:
data = pd.read_csv("SnakegameDataset2_1000_1000.csv",index_col=0)

In [19]:
data.head()

,IsLeftObstacle,IsLeftLeftObstacle,IsForwardObstacle,IsForwardForwardObstacle,IsRightObstacle,IsRightRightObstacle,AngleWithFood,Direction,Decision
0,False,False,False,False,False,False,-0.250000,1,0
1,False,False,False,False,False,True,0.243505,0,0
2,False,False,False,False,False,False,0.237274,1,1
3,False,False,False,False,False,True,0.730536,0,1
4,False,False,False,False,False,False,0.723535,1,1


In [20]:
X = data.drop(["Decision"],axis=1)
Y = data["Decision"]

In [21]:
import numpy as np
np.sum(X.isnull(),axis=0)

IsLeftObstacle              0
IsLeftLeftObstacle          0
IsForwardObstacle           0
IsForwardForwardObstacle    0
IsRightObstacle             0
IsRightRightObstacle        0
AngleWithFood               0
Direction                   0
dtype: int64